### IMPORT LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# scikit-learn
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split



# evaluation metrics
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

# scaling
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# autoencoder
import tensorflow as tf
from tensorflow import keras

# deep learning
from keras.models import Model, Sequential
from keras import regularizers
from keras.layers import (
    Input,
    Conv1D,
    MaxPooling1D,
    Dense,
    Flatten,
    Dropout,
    BatchNormalization,
)
from keras.optimizers import Adam
import tensorflow 
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras import models,layers,activations,losses,optimizers,metrics
from tensorflow.keras.callbacks import EarlyStopping

### LOADING DATASET

In [2]:
credit_card = pd.read_excel("card transactions.xlsx")

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

### DATA PREPROCESSING 

In [ ]:
credit_card.shape

In [ ]:
credit_card.info()

### EXTRACT DAY AND MONTH FOR FURTHER ANALYSIS

In [ ]:
day =pd.to_datetime(credit_card['Date']).dt.dayofweek
month =pd.to_datetime(credit_card['Date']).dt.month
credit_card.insert(1, 'day', day)
credit_card.insert(2, 'month', month)

### DELETING IRRELEVANT COLUMNS

In [ ]:
credit_card = credit_card.drop('Date', axis = 1)
credit_card = credit_card.drop('Recnum', axis = 1)
credit_card = credit_card.drop('Transtype', axis = 1)
credit_card = credit_card.drop('Merch zip', axis = 1)

### CHECKING FOR NULL VALUES

In [ ]:
credit_card.isnull().sum()

### DROP THE NULL VALUES

In [ ]:
credit_card['Merchnum'] = credit_card['Merchnum'].dropna()
credit_card['Merch state'] = credit_card['Merch state'].dropna()

In [ ]:
credit_card.isnull().sum()

In [ ]:
credit_card[['day', 'month','Amount']].describe()

### UNIQUE VALUES

In [ ]:
dict_unique = {}
for i in credit_card.columns.to_list():
    dict_unique[i]=len(credit_card[i].unique())
dict_unique

### CHECKING FOR DUPLICATES

In [ ]:
credit_card.duplicated().sum()

### STATISTICS SUMMARY

In [ ]:
credit_card.describe()

### EXPLORING THE DATASET

In [ ]:
categoricalVar = credit_card.columns.tolist()
for col in categoricalVar:
    df = pd.DataFrame(credit_card[col].value_counts().sort_values(ascending=False).head(20))
    df.plot(kind='bar',figsize=(12,6))
    plt.xlabel(col,fontsize=15)
    plt.ylabel('Frequency',fontsize=15)
    plt.xticks(fontsize=12)

### CORRELATION ANALYSIS

In [ ]:
plt.title('Pearson Correlation Matrix')
sns.heatmap(credit_card.corr(),linewidths=0.25,vmax=0.7,square=True,cmap="winter",
            linecolor='w',annot=True);

The correlation plot shows no presence of multi collinearity amongst the features. All of these features does not show high correlation with the target class

### DISTRIBUTION OF TARGET CLASS

In [ ]:
plt.figure(figsize=(10,6))
labels = ['Not Fraud' , 'Frauds']
explode = [.01,.01]
color = ['skyblue' , 'Red']
sizes = credit_card.Fraud.value_counts().values

plt.pie(sizes,explode,labels,autopct="%1.1f%%", colors = color)
plt.show()

The pie chart shows highly imbalanced dataset. 

In [ ]:
ax=sns.histplot(x='Amount',data=credit_card[credit_card.Amount<=1000],
                hue='Fraud',stat='percent',multiple='dodge',common_norm=False,bins=25)
ax.set_ylabel('Percentage in Each Type')
ax.set_xlabel('Transaction Amount ')
plt.legend(title='Type', labels=['Fraud', 'Not Fraud'])

### STATE VS FRAUD : let's also explore which geographies are more prone to fraud. 

In [ ]:
a=credit_card['Merch state'][credit_card.Fraud==0].value_counts(normalize=True)
a=a.to_frame()
a=a.reset_index()
a.columns = ['Merch state', 'Per']

b=credit_card['Merch state'][credit_card.Fraud==1].value_counts(normalize=True)
b=b.to_frame()
b=b.reset_index()
b.columns = ['Merch state', 'Per']

merged=a.merge(b,on='Merch state')
merged['diff']=merged['Per_y']-merged['Per_x']
merged['diff']=merged['diff']*100
merged=merged.sort_values('diff',ascending=False)

ax1=sns.barplot(data=merged, x='diff',y='Merch state')
ax1.set_xlabel('Percentage Difference')
ax1.set_ylabel('Merch state')
plt.title('The Percentage of Fraudulent over Non-Fraudulent Transcations in Each State')

### Monthly trends

In [ ]:
#month vs fraud
ax=sns.histplot(data=credit_card, x="month", hue="Fraud", common_norm=False,stat='percent',multiple='dodge')
ax.set_ylabel('Percentage')
ax.set_xlabel('Month')
plt.xticks(np.arange(1,13,1))
ax.set_xticklabels(["Jan","Feb","Mar","Apr","May","Jun","Jul",'Aug','Sep','Oct','Nov','Dec'])
plt.legend(title='Type', labels=['Fraud', 'Not Fraud'])

### Daily trend

In [ ]:
ax=sns.histplot(data = credit_card, x="day", hue="Fraud", common_norm=False,stat='percent',multiple='dodge')
ax.set_xticklabels(['',"Mon","Tue","Wed","Thu","Fri","Sat","Sun"])
ax.set_ylabel('Percentage')
ax.set_xlabel('Day')
plt.legend(title='Type', labels=['Fraud', 'Not Fraud'])

### ENCODING INDEPENDENT VARIABLE

In [ ]:
le = LabelEncoder()
credit_card["Merchnum"] = le.fit_transform(credit_card['Merchnum'])
credit_card["Merch description"] = le.fit_transform(credit_card['Merch description'])
credit_card["Merch state"] = le.fit_transform(credit_card['Merch state'])
credit_card

In [ ]:
credit_card.info()

### FEATURE SCALING 

In [ ]:
# Scale amount by log
# Adding a small amount of 0.0001 to amount as log of zero is infinite.
credit_card['amount_log'] = np.log(credit_card.Amount + 0.0001)
credit_card['Cardnum_log'] = np.log(credit_card.Cardnum + 0.0001)
credit_card['Merchnum_log'] = np.log(credit_card.Merchnum + 0.0001)
credit_card['Merch description_log'] = np.log(credit_card['Merch description'] + 0.0001)
credit_card['Merch state_log'] = np.log(credit_card['Merch state'] + 0.0001)
credit_card

In [ ]:
ss = StandardScaler() # object of the class StandardScaler ()
credit_card['amount_scaled'] = ss.fit_transform(credit_card['Amount'].values.reshape(-1,1))
credit_card['Cardnum_scaled'] = ss.fit_transform(credit_card['Cardnum'].values.reshape(-1,1))
credit_card['Merchnum_scaled'] = ss.fit_transform(credit_card['Merchnum'].values.reshape(-1,1))
credit_card['Merch description_scaled'] = ss.fit_transform(credit_card['Merch description'].values.reshape(-1,1))
credit_card['Merch state_scaled'] = ss.fit_transform(credit_card['Merch state'].values.reshape(-1,1))

credit_card

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mm = MinMaxScaler() # object of the class StandardScaler ()
credit_card['amount_minmax'] = mm.fit_transform(credit_card['Amount'].values.reshape(-1,1))
credit_card['Cardnum_minmax'] = mm.fit_transform(credit_card['Cardnum'].values.reshape(-1,1))
credit_card['Merchnum_minmax'] = mm.fit_transform(credit_card['Merchnum'].values.reshape(-1,1))
credit_card['Merch description_minmax'] = mm.fit_transform(credit_card['Merch description'].values.reshape(-1,1))
credit_card['Merch state_minmax'] = mm.fit_transform(credit_card['Merch state'].values.reshape(-1,1))

In [ ]:
#Feature engineering to a better visualization of the values

# Let's explore the Amount by Class and see the distribuition of Amount transactions
fig , axs = plt.subplots(nrows = 1 , ncols = 4 , figsize = (16,4))

sns.boxplot(x ="Fraud",y="Amount",data=credit_card, ax = axs[0])
axs[0].set_title("Fraud vs Amount")

sns.boxplot(x ="Fraud",y="amount_log",data=credit_card, ax = axs[1])
axs[1].set_title("Fraud vs Log Amount")

sns.boxplot(x ="Fraud",y="amount_scaled",data=credit_card, ax = axs[2])
axs[2].set_title("Fraud vs Scaled Amount")

sns.boxplot(x ="Fraud",y="amount_minmax",data=credit_card, ax = axs[3])
axs[3].set_title("Fraud vs Min Max Amount")

# fig.suptitle('Amount by Class', fontsize=20)
plt.show()


In [ ]:
fig , axs = plt.subplots(nrows = 1 , ncols = 4 , figsize = (16,4))

sns.boxplot(x ="Fraud",y="Cardnum",data=credit_card, ax = axs[0])
axs[0].set_title("Fraud vs Cardnum")

sns.boxplot(x ="Fraud",y="Cardnum_log",data=credit_card, ax = axs[1])
axs[1].set_title("Fraud vs Cardnum_log")

sns.boxplot(x ="Fraud",y="Cardnum_scaled",data=credit_card, ax = axs[2])
axs[2].set_title("Fraud vs Cardnum_scaled")

sns.boxplot(x ="Fraud",y="Cardnum_minmax",data=credit_card, ax = axs[3])
axs[3].set_title("Fraud vs Cardnum_minmax")

# fig.suptitle('Amount by Class', fontsize=20)
plt.show()


In [ ]:
fig , axs = plt.subplots(nrows = 1 , ncols = 4 , figsize = (16,4))

sns.boxplot(x ="Fraud",y="Merchnum",data=credit_card, ax = axs[0])
axs[0].set_title("Fraud vs Amount")

sns.boxplot(x ="Fraud",y="Merchnum_log",data=credit_card, ax = axs[1])
axs[1].set_title("Class vs Log Amount")

sns.boxplot(x ="Fraud",y="Merchnum_scaled",data=credit_card, ax = axs[2])
axs[2].set_title("Class vs Scaled Amount")

sns.boxplot(x ="Fraud",y="Merchnum_minmax",data=credit_card, ax = axs[3])
axs[3].set_title("Class vs Min Max Amount")

# fig.suptitle('Amount by Class', fontsize=20)
plt.show()

We can see a slight difference in the log amount of our two Classes

In [ ]:
df = credit_card[['day', 'month', 'Cardnum_minmax','Merchnum_log','Merch description_log', 'Merch state_log', 'amount_log','Fraud']]
df

In [ ]:
#splitting the dataset
X_train, X_test = train_test_split(df, test_size=0.2, random_state=42)
X_train = X_train[X_train.Fraud == 0]
X_train = X_train.drop(['Fraud'], axis=1)

y_test = X_test['Fraud']
X_test = X_test.drop(['Fraud'], axis=1)

X_train = X_train
X_test = X_test
X_train.shape
X_test.shape

### UNDERCOMPLETE AUTOENCODER

The Autoencoder model for anomaly detection has some steps as follows: 

Step 1 is the encoder step. The essential information is extracted by a neural network model in this step.

Step 2 is the decoder step. In this step, the model reconstructs the data using the extracted information.

Step 3: Iterate step 1 and step 2 to adjust the model to minimize the difference between input and reconstructed output, until we get good reconstruction results for the training dataset.

Step 4: Make predictions on a dataset that includes outliers.

Step 5: Set up a threshold for outliers/anomalies by comparing the differences between the autoencoder model reconstruction value and the actual value.

Step 6: Identify the data points with the difference higher than the threshold to be outliers or anomalies.

In [ ]:
# 'tanh' is used for the activation function of the encoded network 
# because it has big learning steps and results in strong or higher gradients

#relu is used on the bottle neck (latent low representation) 
#because model performance is better when trained with relu

#sigmoid is used for binary classification and since we have 
#fraud and non fraudclass then it is good for this problem

#In the input layer, we specified the shape of the dataset. 

In [ ]:
n_features = (X_train.shape[1])
encoder = models.Sequential(name='encoder')
encoder.add(layer=layers.Dense(units=200, activation=activations.tanh, activity_regularizer=regularizers.l1(1e-3),input_shape=[n_features]))
encoder.add(layer=layers.Dense(units=100, activation=activations.tanh))
encoder.add(layer=layers.Dense(units=5, activation=activations.relu))

decoder = models.Sequential(name='decoder')
decoder.add(layer=layers.Dense(units=100, activation=activations.tanh, input_shape=[5]))
decoder.add(layer=layers.Dense(units=200, activation=activations.tanh))
decoder.add(layer=layers.Dense(units=n_features, activation=activations.sigmoid))

autoencoder = models.Sequential([encoder, decoder])

autoencoder.compile(loss=losses.MSE,optimizer=optimizers.Adam(),metrics=[metrics.mean_squared_error])

In [ ]:
# compiling the auto encoder model  with the optimizer of adam and the loss of mse (Mean squared Error).
#autoencoder.compile(optimizer="adam", loss="mse", metrics=['AUC'])

#When fitting the autoencoder model, we can see that the input and output datasets are the same,
#which is the dataset that contains only the normal data points.

#The epochs of 50 and batch_size of 32 mean the model uses 32 datapoints to update the weights in each iteration, 
#and the model will go through the whole training dataset 50 times.

#shuffle=True will shuffle the dataset before each epoch.

# training the auto encoder model on non fraud data 
history = autoencoder.fit(X_train, X_train,batch_size=128,epochs=10,shuffle=True,validation_data=(X_test, X_test))

In [ ]:
# Loss curve

plt.plot(history.history["loss"], "black", linewidth=2.0)
plt.plot(history.history["val_loss"], "green", linewidth=2.0)
plt.legend(["Training Loss", "Validation Loss"], fontsize=14)
plt.xlabel("Epochs", fontsize=10)
plt.ylabel("Loss", fontsize=10)
#plt.ylim([0,1])
plt.title("Loss Curves", fontsize=12)

In [ ]:
#we have an autoencoder model, let's use it to predict the outliers.
#Firstly, we use .predict to get the reconstruction value for the testing data 
#set containing the usual data points and the outliers.


# Predict anomalies/outliers in the training dataset
prediction = autoencoder.predict(X_test)

In [ ]:
mse = np.mean(np.power(X_test - prediction, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse,
                        'true_class': y_test})

In [ ]:
error_df

In [ ]:
error_df.describe()

In [ ]:
# Get the mean squared error between actual and reconstruction/prediction
prediction_loss = tf.keras.losses.mse(prediction, X_test)
prediction_loss

In [ ]:
# Check the prediction loss threshold for 50% of outliers
loss_threshold = np.percentile(prediction_loss, 50)
print(f'The prediction loss threshold for 50% of outliers is {loss_threshold:.2f}')

In [ ]:
#Reconstruction error with fraud

fig = plt.figure()
ax = fig.add_subplot(111)
fraud_error_df = error_df[error_df['true_class'] == 1.0]
_ = ax.hist(fraud_error_df.reconstruction_error.values, bins=10)

In [ ]:
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support)

In [ ]:
fpr, tpr, thresholds = roc_curve(error_df.true_class, error_df.reconstruction_error)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.001, 1])
plt.ylim([0, 1.001])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
precision, recall,th= precision_recall_curve(error_df.true_class, error_df.reconstruction_error)
plt.plot(recall, precision, 'b', label='Precision-Recall curve')
plt.title('Recall vs Precision')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

In [ ]:
plt.plot(th, precision[1:], 'b', label='Threshold-Precision curve')
plt.title('Precision for different threshold values')
plt.xlabel('Threshold')
plt.ylabel('Precision')
plt.show()

In [ ]:
plt.plot(th, recall[1:], 'b', label='Threshold-Recall curve')
plt.title('Recall for different threshold values')
plt.xlabel('Reconstruction error')
plt.ylabel('Recall')
plt.show()

In [ ]:
threshold = 27.56

In [ ]:
pred_y = [1 if e > threshold else 0 for e in error_df.reconstruction_error.values]

In [ ]:
predictions = pd.DataFrame({'true': error_df.true_class,
                           'predicted': pred_y})

In [ ]:
conf_matrix = confusion_matrix(error_df.true_class, pred_y)
conf_matrix
plt.figure(figsize=(12, 12))
LABELS = ['Normal', 'Fraud']
sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Confusion matrix")
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

In [ ]:
# Check the prediction performance
print(classification_report(y_test, pred_y))